<a href="https://colab.research.google.com/github/MikolajKasprzyk/data_science_bootcamp/blob/main/05_uczenie_maszynowe_sklearn/02_metryki_klasyfikacja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

#### <a name='a1'></a>  Metryki - Klasyfikacja binarna - dwie klasy do przewidzenia
#### <a name='a2'></a> Accuracy - Dokładność klasyfikacji

####   $$Accuracy = \frac{correct\ predictions}{total\ predictions} * 100$$

In [ ]:
# przykładowe wyniki testowe i predicted z modelu
y_true = np.array([1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1])
y_pred = np.array([0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)

0.7241379310344828

In [ ]:
# standarodowo w plotly uzywamy dataframe
results = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})
results

,y_true,y_pred
0,1,0
1,0,0
2,1,1
3,1,1
4,0,0
5,1,1
6,1,0
7,0,0
8,1,1
9,0,1


In [ ]:
results.sort_values(by='y_true', inplace=True)
results.reset_index(inplace=True, drop=True)
results['sample'] = results.index + 1 # numer probki liczone od 1
results

,y_true,y_pred,sample
0,0,0,1
1,0,1,2
2,0,0,3
3,0,0,4
4,0,1,5
5,0,0,6
6,0,0,7
7,0,0,8
8,0,0,9
9,0,0,10


In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=results['sample'], y=results['y_true'], mode='markers', name='y_true'), row=1, col=1)
fig.add_trace(go.Scatter(x=results['sample'], y=results['y_pred'], mode='markers', name='y_pred'), row=2, col=1)
fig.update_layout(width=900, height=500, title='Klasyfikator binarny')
fig.show()

# Macierz konfuzji

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_true, y_pred)

array([[ 8,  2],
       [ 4, 16]])

In [ ]:
import plotly.figure_factory as ff

def plot_confusion_matrix(cm):
    cm = cm[::-1] # to wynika ze specyfiki działania tej funkcij
    cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_1', 'true_0'])

    fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index),
                                      colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(width=400, height=400, title='Confusion Matrix', font_size=16)
    fig.show()

plot_confusion_matrix(confusion_matrix(y_true, y_pred))

In [ ]:
cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel() # true positive, false positive, true negative, false neagative
print(f'TN: {tn}')
print(f'FP: {fp}')
print(f'FN: {fn}')
print(f'TP: {tp}')

TN: 8
FP: 2
FN: 4
TP: 16


In [ ]:
# type I error, false positive rate
fpr = fp / (fp + tn)
fpr

0.2

In [ ]:
# type II error, false negative rate
fnr = fn / (fn + tp)
fpr

0.2

In [ ]:
# precission
precission = tp / (tp + fp)
precission

0.8888888888888888

In [ ]:
# Recall
recall = tp / (tp + fn)
recall

0.8

# Krzywa ROC

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, tresh = roc_curve(y_true, y_pred, pos_label=1)
roc = pd.DataFrame({'fpr':fpr, 'tpr':tpr})
roc

,fpr,tpr
0,0.0,0.0
1,0.2,0.8
2,1.0,1.0


In [ ]:
# gotowa funkcja do tworzenia krzywej ROC
def plot_roc_curve(y_true, y_pred):
    # Binary classification
    from sklearn.metrics import roc_curve
    fpr, tpr, tresh = roc_curve(y_true, y_pred, pos_label=1)
    roc = pd.DataFrame({'fpr':fpr, 'tpr':tpr})

    fig = go.Figure(data=[go.Scatter(x=roc['fpr'], y=roc['tpr'], line_color='red', name='ROC Curve'),
                        go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line_dash='dash', line_color='navy')],
                    layout=go.Layout(xaxis_title='False Positive Rate',
                                    yaxis_title='True Positive Rate',
                                    title='ROC Curve',
                                    showlegend=False,
                                    width=800,
                                    height=400))
    fig.show()
plot_roc_curve(y_true, y_pred)
# im wieksze pole pod wykresem tym lepiej

# <a name='a5'></a> Metryki - Klasyfikacja wieloklasowa

#### <a name='a6'></a> Accuracy - Dokładność klasyfikacji

####   $$Accuracy = \frac{correct\ predictions}{total\ predictions} * 100$$

In [ ]:
y_true = np.array([1, 0, 1, 2, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 2, 1, 1, 2, 2, 1, 0, 1, 1, 0, 2, 1, 1, 2, 2])
y_pred = np.array([0, 0, 1, 2, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1, 0, 2, 1, 0, 1, 1, 1, 2, 2])

accuracy_score(y_true, y_pred)

0.7241379310344828

## Macierz konfuzji


In [ ]:
cm = confusion_matrix(y_true, y_pred)
cm

array([[ 6,  1,  0],
       [ 3, 10,  2],
       [ 0,  2,  5]])

In [ ]:
def plot_confusion_matrix(cm):
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1', 'pred_2'], index=['true_2','true_1', 'true_0'])

    fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index),
                                      colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(width=400, height=400, title='Confusion Matrix', font_size=16)
    fig.show()

plot_confusion_matrix(cm)

## Raport klasyfikacji

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, target_names=['label_1', 'label_2', 'label_3']))

              precision    recall  f1-score   support

     label_1       0.67      0.86      0.75         7
     label_2       0.77      0.67      0.71        15
     label_3       0.71      0.71      0.71         7

    accuracy                           0.72        29
   macro avg       0.72      0.75      0.73        29
weighted avg       0.73      0.72      0.72        29

